In [1]:
%env CUDA_VISIBLE_DEVICES=5,6,7
import os

import itertools
import pickle
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import math 

from torchtext import data, datasets
from pathlib import Path

import sys
sys.path.append('../')
%matplotlib inline

env: CUDA_VISIBLE_DEVICES=5,6,7


In [2]:
torch.cuda.device_count()

3

In [3]:
# Init cuda
#device = "cuda:6" if torch.cuda.is_available() else "cpu"
#idevice = 6 if torch.cuda.is_available() else -1
#torch.cuda.set_device(idevice)
#device, idevice

In [4]:
TEXT = data.Field()
PATH = Path('./wikitext/')

In [5]:
batch_size = 1024
sequence_length = 10
grad_clip = 0.1
lr = 4.
best_val_loss = None
log_interval = 10_000

In [6]:
from torchtext.datasets.language_modeling import LanguageModelingDataset

class WikiTextRu(LanguageModelingDataset):

    urls = ['http://files.deeppavlov.ai/datasets/wikitext_ru.zip']
    name = 'wikitext_ru'
    dirname = 'wikitext_ru'

    @classmethod
    def splits(cls, text_field, root='.data', 
               train='ru.wiki.train.txt', validation='ru.wiki.valid.txt', test='ru.wiki.test.txt',  **kwargs):

        return super().splits(text_field=text_field, root=root, 
                              train=train, validation=validation, test=test, **kwargs)

    @classmethod
    def iters(cls, batch_size=32, bptt_len=35, device=0, root='.data', vectors=None, **kwargs):
       
        TEXT = data.Field()
        train, val, test = cls.splits(TEXT, root=root, **kwargs)
        TEXT.build_vocab(train, vectors=vectors)

        return data.BPTTIterator.splits((train, val, test), 
                                        batch_size=batch_size, bptt_len=bptt_len, device=device)

In [7]:
from torchtext.vocab import Vectors

class RuFastText(Vectors):

    url_base = 'http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_word_tokenize/ft_native_300_ru_wiki_lenta_nltk_word_tokenize.vec'

    def __init__(self, **kwargs):
        url = self.url_base
        name = os.path.basename(url)
        super().__init__(name, url=url, **kwargs)


In [8]:
%%time
train_ds, valid_ds, test_ds = WikiTextRu.splits(TEXT, root=PATH)

CPU times: user 1min 15s, sys: 18.8 s, total: 1min 33s
Wall time: 1min 33s


In [9]:
ru_vectors = RuFastText()

In [10]:
%%time
TEXT.build_vocab(train_ds, min_freq=30, max_size=10_000, vectors=ru_vectors)

CPU times: user 2min 15s, sys: 14.2 s, total: 2min 29s
Wall time: 2min 29s


In [11]:
ntokens = len(TEXT.vocab)
ntokens

10002

In [12]:
%%time
train_loader, val_loader, test_loader = data.BPTTIterator.splits((train_ds, valid_ds, test_ds), 
                                        batch_sizes=(batch_size, batch_size, batch_size), 
                                        bptt_len=sequence_length, repeat=False) 

CPU times: user 173 µs, sys: 33 µs, total: 206 µs
Wall time: 213 µs


In [13]:
class RNNModel(nn.Module):

    def __init__(self, rnn_type, emb_vectors, nhid, nlayers, dropout=0.5):
        super().__init__()
        ntoken = emb_vectors.shape[0]
        ninp = emb_vectors.shape[1]
        self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.encoder.weight.data.copy_(emb_vectors);
        self.encoder.weight.requires_grad = False
    
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(ninp, nhid, nlayers, dropout=dropout)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(ninp, nhid, nlayers, dropout=dropout)
        # self.rnn = nn.DataParallel(self.rnn)
        self.decoder = nn.Linear(nhid, ntoken)

        self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, x, hidden=None):
        emb = self.drop(self.encoder(x))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()).data
        if self.rnn_type == 'LSTM':
            return (weight.new(self.nlayers, bsz, self.nhid).zero_(),
                    weight.new(self.nlayers, bsz, self.nhid).zero_())
        else:
            return weight.new(self.nlayers, bsz, self.nhid).zero_()

In [14]:
def evaluate(data_loader):
    model.eval()
    total_loss = 0
    ntokens = len(TEXT.vocab)
    hidden = model.init_hidden(batch_size)
    for i, b in enumerate(data_loader):
        data, targets = b.text, b.target
        output, hidden = model(data)
        output_flat = output.view(-1, ntokens)
        total_loss += criterion(output_flat, targets.view(-1)).item()
    return total_loss / len(data_loader)

In [15]:
def train():
    model.train()
    total_loss = 0
    ntokens = len(TEXT.vocab) #len(corpus.dictionary)
    for batch, b in enumerate(train_loader):
        data, targets = b.text, b.target
        model.zero_grad()
        output, hidden = model(data)
        loss = criterion(output.view(-1, ntokens), targets.view(-1))
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm_(filter(lambda p: p.requires_grad, model.parameters()), grad_clip)
        for p in filter(lambda p: p.requires_grad, model.parameters()):
            p.data.add_(-lr, p.grad.data)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_loader), lr, cur_loss, math.exp(cur_loss)))
            total_loss = 0

In [16]:
def generate(n=50, temp=1.):
    model.eval()
    x = torch.rand(1, 1).mul(ntokens).long().cuda()
    hidden = None
    out = []
    for i in range(n):
        output, hidden = model(x, hidden)
        s_weights = output.squeeze().data.div(temp).exp()
        s_idx = torch.multinomial(s_weights, 1)[0]
        x.data.fill_(s_idx)
        s = TEXT.vocab.itos[s_idx]
        out.append(s)
    return ' '.join(out)

In [17]:
emb_vectors = TEXT.vocab.vectors
model = RNNModel('LSTM', emb_vectors, 128, 3, 0.3).cuda()
criterion = nn.CrossEntropyLoss()

In [18]:
%%time
with torch.no_grad():
    print('sample:\n', generate(15), '\n')

sample:
 белый релиза 1832 другими функций перешли севере участвовали оркестром 1875 легенде количеством Швейцария переехал интерес 

CPU times: user 14.3 ms, sys: 0 ns, total: 14.3 ms
Wall time: 13.2 ms


In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
for epoch in range(1, 2):
    %time train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')


| epoch   1 | 10000/37437 batches | lr 4.00 | loss  5.12 | ppl   166.86
| epoch   1 | 20000/37437 batches | lr 4.00 | loss  4.68 | ppl   107.90
| epoch   1 | 30000/37437 batches | lr 4.00 | loss  4.51 | ppl    90.91
CPU times: user 16min 32s, sys: 5min 44s, total: 22min 16s
Wall time: 22min 17s
-----------------------------------------------------------------------------------------
| end of epoch   1 | valid loss  4.27 | valid ppl    71.67
-----------------------------------------------------------------------------------------
sample:
 <unk> <unk> вопрос 22 <unk> <unk> Ирана <unk> По <unk> миру <eos> <unk> государством расположены <unk> <unk> показал в <unk> ; 100 проект был <unk> , хотя метод <unk> государственной " <unk> <unk> " , <unk> был произведён , который занял в состав <unk> <unk> с <unk> отделом их <unk> 



In [21]:
for epoch in range(2, 6):
    %time train()
    val_loss = evaluate(val_loader)
    print('-' * 89)
    print('| end of epoch {:3d} | valid loss {:5.2f} | valid ppl {:8.2f}'.format(
        epoch, val_loss, math.exp(val_loss)))
    print('-' * 89)
    if not best_val_loss or val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
        lr /= 4.0
    with torch.no_grad():
        print('sample:\n', generate(50), '\n')



| epoch   2 | 10000/37437 batches | lr 4.00 | loss  4.37 | ppl    79.07
| epoch   2 | 20000/37437 batches | lr 4.00 | loss  4.32 | ppl    75.53
| epoch   2 | 30000/37437 batches | lr 4.00 | loss  4.29 | ppl    73.32
CPU times: user 16min 33s, sys: 5min 47s, total: 22min 20s
Wall time: 22min 19s
-----------------------------------------------------------------------------------------
| end of epoch   2 | valid loss  4.11 | valid ppl    61.23
-----------------------------------------------------------------------------------------
sample:
 наук и Дэвид государств 2 октября 2014 года. <unk> <unk> <unk> с <unk> на всех 1-й , ранее результате <unk> <unk> Таким образом о своей чемпионатах с войском в национальном <unk> поражений во время <unk> вторжения на основе место. Кубка сезонов за тем , что <unk> <unk> <unk> <unk> <unk> 

| epoch   3 | 10000/37437 batches | lr 4.00 | loss  4.26 | ppl    70.46
| epoch   3 | 20000/37437 batches | lr 4.00 | loss  4.23 | ppl    69.03
| epoch   3 | 30000/37